# Mask R-CNN - Train on OpenImages Dataset

In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

from openimages2019 import setup as st
from openimages2019 import utils as u

from skimage.draw import rectangle

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

DATA_DIR = os.path.join(ROOT_DIR, "../data")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(DATA_DIR, "mask_rcnn_coco.h5")

#Make GPUs visible
!export HIP_VISIBLE_DEVICES=2,3



import mlflow
#add mlflow stuff

MLFLOW_SERVER = 'occ01ap200.na.simplot.com'

os.environ['NO_PROXY'] = MLFLOW_SERVER
mlflow.tracking.set_tracking_uri('http://' + MLFLOW_SERVER + ':5005')
EXPERIMENT_NAME = 'kaggle_openimage_mask_rcnn_v1.0'
mlflow.set_experiment(EXPERIMENT_NAME)

# os.environ['AZURE_STORAGE_ACCESS_KEY'] = ''

Using TensorFlow backend.


In [2]:
classes = st.load_classes()
annotations = st.load_annotations_by_image()

val_data = st.load_dataset(annotations, DATA_DIR, classes, is_train=False)

## Utility Functions

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load dataset and display samples

In [ ]:
from mrcnn import visualize
import numpy as np

# Load and display random samples ... sanity check for data load
image_ids = np.random.choice(val_data.image_ids, 2)
for image_id in image_ids:
    image = val_data.load_image(image_id)
    mask, class_ids = val_data.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, val_data.class_names)

## Train

In [5]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.

def log_params(pz):
    for k,v in pz.items():
        mlflow.log_param(k,v)

        
class InferenceConfig(u.KaggleConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2
    BATCH_SIZE = 10

inf_config = InferenceConfig()


def train(model, train_data, val_data, params):
    with mlflow.start_run():
        
#         model.train(dataset_train, dataset_val, 
#                     learning_rate=params['learn_rate'], 
#                     epochs=params['epochs'], 
#                     layers=params['layers'])

        log_params(params)
        
        #Inference to get mAP
        inf_model = modellib.MaskRCNN(mode="inference", config=inf_config, model_dir=MODEL_DIR)
        model_path = inf_model.find_last()
        inf_model.load_weights(model_path, by_name=True)
        mAP = u.eval_mAP(inf_model, val_data, inf_config, params['mAP_sample_size'])
        
        mlflow.log_metric('mAP', mAP)
        
        return model_path, mAP
        #TODO inference and record score
    
    
# mlflow.log_param('Code_Version_Git_Hash',git_hash)

# predictions = clf.predict(X_test)

# mlflow.log_metric('AUC of ROC', skm.roc_auc_score(y_test,predictions))

# pickle.dump(clf,open(model_filepath,'wb'))
# mlflow.log_artifact(model_filepath)

In [6]:


# params = {'learning_rate' : 0.00001,
#           'epochs' : 5,
#           'layers' : 'all'
#          }

config = u.KaggleConfig()

print("here_00")
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

print("here_01")

init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    model.load_weights(model.find_last(), by_name=True)

    
params = {'learning_rate' : config.LEARNING_RATE,
          'epochs' : 5,
          'layers' : 'heads',
          'mAP_sample_size' : 50,
         }
    
print("here_02")
train(model, None, val_data, params)

here_00
here_01
here_02
hereHere
here
here1
here2
Re-starting from epoch 50
here3
here4
here5


('/home/nbuser/repo/logs/kaggle20190812T2002/mask_rcnn_kaggle_0050.h5',
 0.048665468299930736)